# Gradient calculation for a multitarget dissipative problem (Lindblad's Equation)

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import importlib
import numpy as np
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from qutip import *
from qutip.qip.operations import toffoli

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers
import qocttools.target as target

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

In this notebook we test the computation of the gradient when using a multi-target definition. The formulation is done with the Lindblad's equation, in the presence of dissipation. 

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.15, 0.0, 0.0)$ T

* $\vec{H}_m = (0, 0.001, 0.0)$ T

In [ ]:
#We define the parameters of the Hamiltonian

S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

In [ ]:
# We define the amplitude of the constant and variable magnetic fields, and, with that, we can build
# the unperturbed hamiltonian H0 and the perturbation operator V, which will be multiplied by the pulse f(u,t) later.

H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)

In [ ]:
# We transform both H0 and V to the basis of eigenstates of H0
eigenvalues, eigenstates = H0.eigenstates()
H0 = H0.transform(eigenstates) - eigenvalues[0] #WHY NOT NOW? 
V = V.transform(eigenstates)

The following is a check for the Hermiticiy of the operators. In case they are not exactly Hermitian, they are symmetrized.

In [ ]:
H0 = 0.5 * (H0 + H0.dag())
V = 0.5 * (V + V.dag())

In [ ]:
# We define the Spin and Ladder Operators. This will be useful for us to build the Lindbladian later, since the
# dissipative operators A, A* will be proportional to Sp and Sm, respectively.

Sx = jmat(S, "x")
Sy = jmat(S, "y")
Sz = jmat(S, "z")
Sp = (Sx + (1j*Sy))
Sm = (Sx - (1j*Sy))

In [ ]:
# We compute the transition frequencies

w = np.zeros(dim-1)
taui = np.zeros(dim-1)
for i in range(dim-1):
    w[i] = eigenvalues[i+1] - eigenvalues[i] # Transition frequencies
    taui[i] = 2.0*np.pi/w[i] # Transition periods
    print("Transition {:d}: w = {:f} MHz, tau = {:f} ns".format(i, w[i], 1000.0*taui[i]/(2.0*np.pi)))

In [ ]:
# Time Array Definition: Duration of the pulse and Time Grid

T = 5*taui[0]
#T = 5*taui.max() # Different from gradient-dissipation.ipynb
print("T = {:f} us*2*pi = {:f} ns".format(T, 1000*T/(2.0*np.pi)))
time = math_extra.timegrid(H0, T, 4.0)
print('# Time steps =', time.shape[0])

# Target Operator and Target States

In [ ]:
# We choose a toffoli gate as our target, but it could be any.
U_target = Qobj(toffoli().data)

We will have N=8 initial states $|j\rangle$, whose corresponding target states will be $|j'\rangle=U|j\rangle$.

In [ ]:
ket_ini = []
ket_target = []
rho_ini = []
rho_target = []

for i in range (0, dim):
    ket_ini.append(basis(dim,i))
    ket_target.append(U_target*ket_ini[i])

    rho_ini.append(fock_dm(dim,i))
    rho_target.append(U_target.dag()*rho_ini[i]*U_target)
    
rho=rho_ini #This will be the array of states that we will propagate.

# Control function

The control function is parametrized with the Fourier expansion as follows:
\begin{equation}
    f(u, t) = \frac{1}{\sqrt{T}}u_0 + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k}\cos(\omega_kt) + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k + 1}\sin(\omega_kt),
\end{equation}
where $u_0\dots u_{2M + 1}$ are the control parameters. This way, we can compute the derivate respect any control parameter as
\begin{equation}
    \frac{\partial f}{\partial u_m}(u, t) = f(e_m, t),
\end{equation}
where $e_m$ is the set of parameters where all of them are zero except the m-th ane, that is equal to one.

This pulse parametrization is included in the typical_pulses.py file as pulse class.

In [ ]:
M = 10 #Number of parameters

# We will not take into account the frequency limitations of our experimental display. The change that would introduce is
# not qualitatively relevant, and it would steeply increase the number of parameters, making the calculations way longer.

omega = np.zeros(M+1)
omega[0] = 0.0
for k in range(1, M+1):
    omega[k] = (2.0*np.pi/T) * k
    print("omega[{:d}] = {:f} MHz".format(k, omega[k]))

# u = 1.0*np.random.rand((2*M + 1))
u = np.zeros(2*M+1)
u[2] = 1.0 # Arbitrary initial set of parameters
u[3] = 1.0
f = pulses.pulse("fourier", T, u = u)


# print("Amplitudes:")
# print("{:f} mT".format(u[0]/np.sqrt(T)))
# for m in range(1, 2*M+1):
#     print("{:f} mT".format(2.0*u[m]/(np.sqrt(T)))) # Proceso inverso. No es A -> Tmax, sino Tmax=T -> A


In [ ]:
# We plot the pulse

fig, ax = plt.subplots()

ax.plot(time * 1000/(2.0*np.pi), f.fu(time, u))
ax.set_xlabel("Time (ns)")
ax.set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
ax.set_ylabel("f(t) (mT)")
if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

# Gradient calculation for multi-target state-to-state transitions, with dissipation

In [ ]:
A = 5.0*Sp

In [ ]:
H = hamiltonians.hamiltonian(H0, [V], [A])

In [ ]:
tg = target.Target('expectationvalue', operator = rho_target)

opt = qoct.Qoct(H, T, time.shape[0], tg, f, rho_ini,
                interaction_picture = True,
                solve_method = 'cfmagnus4')

In [ ]:
derqoct, dernum, error = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))
data.append(derqoct)

# Datafile

In [ ]:
with open("data", "w") as f:
    for i in data:
        f.write("{:.14e}\n".format(i))